In [ ]:
import os,cv2
os.sys.path.append("../utilities/")


In [2]:
import numpy as np

import support_utility_openvino
from mobileface_sort_v1 import Sort


/home/prateek/.virtualenvs/openvino/lib/python3.6/site-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


In [3]:
detection_model_path = '/media/prateek/prateek_space/intel_model_dump/intel/face-detection-adas-0001/FP32/face-detection-adas-0001.xml'
device = 'CPU'
output_support = support_utility_openvino.create_plot()

In [4]:
face_detection = support_utility_openvino.async_infer(4)
face_detection.load_model(model_path=detection_model_path,device=device)

Available Devices :  ['CPU', 'GNA', 'GPU']
OPTIMIZATION_CAPABILITIES for CPU:  ['FP32', 'FP16', 'INT8', 'BIN']
model inputs : dict_keys(['data'])
model outputs :  dict_keys(['detection_out'])


In [5]:
channel = "/media/prateek/prateek_space/dataset/veraval_production/videos/face_veraval/202003211445.mp4"

In [6]:
cap = cv2.VideoCapture(channel)
ret,frame = cap.read()
output_support.get_resolution_thresh(frame)

(0.002951388888888889, 0.07165316358024691)

In [6]:
mot_tracker = Sort(10, 3)

thresh = .2

cap = cv2.VideoCapture(channel)
frame_width, frame_height = int(cap.get(3)),int(cap.get(4))
# ratio_li = []
write_video = False
if write_video:
    fourcc = cv2.VideoWriter_fourcc(*'X264')
    videowriter = cv2.VideoWriter("output_vids/face_detection/"+"temperature_reading"+".mp4",fourcc, 30, (int(cap.get(3)),int(cap.get(4))))
face_detection.reinit_model()
while True:

    ret,frame = cap.read()
    if not ret:
        break
    face_detection.predict(frame)
    if face_detection.frame_processed>=face_detection.num_requests:
        frame,attr,res = face_detection.postprocess_op()
        res_filt,bboxes = output_support.res2bbox(res[0],(frame_width,frame_height),[.0005,.06],thresh = thresh,return_res=True)
        if len(bboxes)>0:
            ids = [[0]]*len(bboxes)
            dets = np.concatenate((bboxes,ids),axis=1)
            trackers = mot_tracker.update(dets)
            for d in trackers:
                cv2.rectangle(frame, (int(d[0]), int(d[1])), (int(d[2]), int(d[3])), (255,255,255), 3)
                cv2.putText(frame, str('%s%d' % ('face', d[4])), 
                           (int(d[0]), int(d[1]) - 5), cv2.FONT_HERSHEY_COMPLEX , 0.8, (255,255,255), 2)
            cv2.imshow("preview",frame)
            k = cv2.waitKey(1)
            if k==ord('q'):
                break
cap.release()
cv2.destroyAllWindows()

/home/prateek/.virtualenvs/openvino/lib/python3.6/site-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
